# Importação das bibliotecas utilizadas:

* `pandas` biblioteca utilizada para tabulação e tratamento de dados
* `googletrans` biblioteca não oficial utilizada para tradução automática utilizando o Google Translate
* `numpy` biblioteca utilizada para agilizar e otimizar operações numéricas durante o processamento
* `re` biblioteca utilizada para busca de expressões regulares e tratamento de strings (texto)
* `json` biblioteca utilizada para geração do JSON que abastece as visualizações (_front end_)
* `pymed` biblioteca utilizada para busca e coleta automatizada de artigos científicos da base PubMED
* `vaderSentiment` biblioteca com modelos pré-treinados para classificação de texto (sentiment analysis)

In [8]:
import pandas as pd
from googletrans import Translator
import numpy as np
import re
import json
from pymed import PubMed
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Passo 1 - Inicialização do pymed e pesquisa inicial 

In [9]:
email = 'fcc-pesquisas@hotmail.com'
query = '(Sars-cov2 OR COVID OR nCOV)'
#query = 'Monkeypox'
pubmed = PubMed(tool="PyMed", email="fcc-pesquisas@hotmail.com")
n = pubmed.getTotalResultsCount(query=query)
results = pubmed.query(query, max_results=99999999999)
print(n)

284609


# Passo 2 - Carregamento da base de dados de fármacos, qualis e H-Index

Um dos pilares da solução proposta é a identificação dos fármacos citados e a classificação das fontes segundo seu impacto e qualidade. Para isso, utilizamos a última atualização do qualis (avaliação de 2019) e o H-Index, segundo a plataforma [Scimago](https://www.scimagojr.com/journalrank.php). 

Para a identificação dos fármacos, utilizamos os nomes comerciais e nomes de princípios ativos de milhares de medicamentos aprovados pela FDA e também aqueles listados no DrugCentral. Para a maioria dos medicamentos, também são verificadas suas abreviações (ex: HCQ = Hydroxichloroquine, TCZ = Tocilizumabe, etc.). Além destes medicamentos, foram adicionados manualmente os termos `vaccine` e  `convalescent plasma`, pois se tratam de termos mais genéricos, mas que se encaixam no escopo do projeto aqui desenvolvido.

In [85]:
# Passo 2 - Carregar base de dados de fármacos, qualis e H-Index
with open('Dados/drug-drugsfda-0001-of-0001.json') as file:
    fda = json.load(file)
to_lower = lambda x: x.lower()
farmacos = pd.read_csv('Dados/DrugCentral/drug.target.interaction.tsv.gz',sep='\t')
farm = [f.lower() for f in pd.unique(farmacos['DRUG_NAME'])]
druglist = []
for entry in fda['results']:
    if 'products' in entry.keys():
        drugs = entry['products']
        for d in drugs:
            names = [d['brand_name']]
            for ig in d['active_ingredients']:
                names.append(ig['name'])
    druglist.extend(names)
druglist = [d.lower() for d in pd.unique(druglist)]
farm.extend(['vaccine','convalescent plasma'])
farm.extend(druglist)
farm.remove('0')
farm = pd.unique(farm)
qualis = pd.read_csv('Dados/qualis2019.csv', delimiter=';')
qualis['TITULO'] = qualis['TITULO'].apply(to_lower)
sjr = pd.read_csv('Dados/scimago.csv',delimiter=';')
sjr['Title'] = sjr['Title'].apply(to_lower)

# Passo 3 - Tabulação dos resultados, tradução e classificação

Nesta etapa, partimos dos resultados retornados pela API `pymed`, e os processamos, transformando-os em uma tabela contendo resumo, resumo traduzido, metadados da publicação, extrato qualis da publicação, H-index, fármacos citados favoravelmente e fármacos citados desfavoravelmente. Esta última classificação é realizada utilizando _sentiment analysis_ utilizando, por hora, um modelo pré-treinado.

É importante ressaltar que, como estamos tratando com dados textuais, todos os textos foram convertidos para minúsculo. Pontuações também foram removidas para evitar confusão durante a fase de _sentiment analysis_.

In [ ]:
translator = Translator(service_urls=['translate.google.com'])
analyzer = SentimentIntensityAnalyzer()
base = dict.fromkeys(['titulo', 'resumo','resumo_traduzido', 'revista', 'data', 'qualis', 'H-index','doi', 'tipo'],[np.nan]*n) | dict.fromkeys(['pro', 'contra', 'indefinido'],[np.nan]*n) 
base = pd.DataFrame(base)
k = 0
for r in results:
    base.at[k,'titulo'] = r.title
    try:
        base.at[k,'resumo'] = r.abstract
    except:
        continue
    try:
        base.at[k,'doi'] = r.doi
    except:
        pass
    try:
        translation = translator.translate(r.abstract, dest='pt')
        base.at[k,'resumo_traduzido'] = translation.text
    except:
        pass
    try:
        base.at[k,'resultados'] = r.results
    except:
        pass
    try:
        base.at[k,'revista'] = r.journal
    except:
        pass
    try:
        base.at[k, 'data'] = r.publication_date
    except:
        pass
    try:
        base.at[k, 'H-index'] = sjr.loc[sjr['Title']==r.journal.lower(),'H index'].values[0]
    except:
        pass
    try:
        base.at[k, 'qualis'] = qualis.loc[qualis['TITULO']==r.journal.lower(),'ESTRATO'].values[0]
    except:
        pass
    if isinstance(r.abstract, str):
        nota = {}
        usados = []
        if isinstance(r.title, str):
            texto = to_lower(' '.join([r.title,r.abstract]))
        else:
            texto = to_lower(r.abstract)
        if 'systematic review' in texto:
            base.at[k,'tipo'] = 'revisão sistemática'
        elif 'meta analysis' in texto or 'meta-analysis' in texto:
            base.at[k,'tipo'] = 'meta análise'
        elif 'case report' in texto:
            base.at[k,'tipo'] = 'estudo de caso'
        elif 'cohort' in texto:
            base.at[k,'tipo'] = 'estudo de coorte'
        elif 'clinical trial' in texto:
            base.at[k,'tipo'] = 'estudo clínico randomizado'
        else:
            base.at[k,'tipo'] = 'outros'
        frases = re.split('\; |\.|\n',texto)
        for i in range(len(frases)):
            palavras = frases[i].replace('\n', ' ').replace('.', '').replace(',','').split()
            for j in range(len(palavras)):
                t = palavras[j]
                if (t in farm):
                    if t not in usados:
                        usados.append(t)
                        nota[t] = 0
                    nota[t] += analyzer.polarity_scores(frases[i])['compound']
        for t in usados:
            if nota[t] > 0.25:
                if isinstance(base.at[k,'pro'], list):
                    base.at[k,'pro'].append(t)
                else:
                    base.at[k,'pro'] = [t]
            elif nota[t] < -0.25:
                if isinstance(base.at[k,'contra'], list):
                    base.at[k,'contra'].append(t)
                else:
                    base.at[k,'contra'] = [t]
            else:
                if isinstance(base.at[k,'indefinido'], list):
                    base.at[k,'indefinido'].append(t)
                else:
                    base.at[k,'indefinido'] = [t]
    else:
        base.at[k,'tipo'] = 'indefinido'
    k +=1
    if k > n:
        break
    text = np.round(100*k/n,2)
    print(f'{"|"*int(text) + "-"*(100-int(text))}  {text}%', end='\r')
print(f'{"|"*100} 100.00%')
base.to_csv('Pymed_CVD.csv')
valid = base.loc[(~base['pro'].isnull()) | (~base['contra'].isnull()) | (~base['indefinido'].isnull())]
valid.to_csv('Pymed_validos.csv')

# Passo 4 - Processamento dos resultados para geração do dashboard

Nesta etapa, os resultados tabulados são processados para alimentar o dashboard. Aqui são contados os arquivos que descrevem positivamente ou negativamente os resultados encontrado. A classificação do tipo de estudo é realizada diretamente no _front end_, permitindo a visualização de um dashboard com filtros.

In [219]:
pro_list = []
con_list = []
ind_list = []
for i in range(len(valid)):
    if isinstance(valid.at[i,"pro"], list): 
        pro_list.extend(valid.at[i,"pro"])
    elif isinstance(valid.at[i,"pro"], str): 
        pro_list.extend(eval(valid.at[i,"pro"]))
    if isinstance(valid.at[i,"contra"], list):
        con_list.extend(valid.at[i,"contra"])
    elif isinstance(valid.at[i,"contra"], str):
        con_list.extend(eval(valid.at[i,"contra"]))
    if isinstance(valid.at[i,"indefinido"], list):
        ind_list.extend(valid.at[i,"indefinido"])
    elif isinstance(valid.at[i,"indefinido"], str):
        ind_list.extend(eval(valid.at[i,"indefinido"]))

In [220]:
tecnologias = pro_list.copy()
tecnologias.extend(con_list)
tecnologias.extend(ind_list)
tecnologias = pd.unique(tecnologias)
tec = {"tecnologia":[], "pro":[], "contra":[]}
for t in tecnologias:
    tec["tecnologia"].append(t)
    tec["pro"].append(pro_list.count(t))
    tec["contra"].append(con_list.count(t) + ind_list.count(t))

In [221]:
tec = pd.DataFrame(tec)
tec = tec.sort_values(by=['pro','contra'],ascending = [False,True])
tec = tec.loc[(tec["pro"]>1)|(tec["contra"]>1)]
tec.transpose().to_csv("cvd_pymed.csv",sep=";",index=False, header=False)